In [1]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv('scraped_flight_data.csv')

In [5]:
df.head()

,Airline,Source,Destination,Departure,Arrival,Number of Stops,Stopover Details,Price,Class,Date
0,"Delta • Operated by Virgin Atlantic, KLM",Toronto Pearson Intl,Bengaluru Intl,10:40 pm,2:45 am+3,2 stops,", 19h 25m layover, <b>Amsterdam Schiphol</b>, ...","C$ 2,229",Main Cabin,2024-06-22
1,"Delta • Operated by Virgin Atlantic, KLM",Toronto Pearson Intl,Bengaluru Intl,10:40 pm,2:45 am+3,2 stops,", 20h 35m layover, <b>Amsterdam Schiphol</b>, ...","C$ 2,229",Main Cabin,2024-06-22
2,"Delta • Operated by Virgin Atlantic, KLM",Toronto Pearson Intl,Bengaluru Intl,10:40 pm,2:45 am+3,2 stops,", 22h 20m layover, <b>Amsterdam Schiphol</b>, ...","C$ 2,524",Main Cabin,2024-06-22
3,"Delta • Operated by KLM, Air France",Toronto Pearson Intl,Bengaluru Intl,10:40 pm,11:55 pm+2,2 stops,", 4h 55m layover, <b>Amsterdam Schiphol</b>, ,...","C$ 3,690",Main Cabin,2024-06-22
4,"Delta • Operated by KLM, Air France",Toronto Pearson Intl,Bengaluru Intl,10:40 pm,11:55 pm+2,2 stops,", 2h 10m layover, <b>Amsterdam Schiphol</b>, ,...","C$ 3,690",Main Cabin,2024-06-22


In [7]:
# Define a function to clean and split the data
def clean_split_stopover(details):
    if isinstance(details, str):
        # Split the details by the delimiter ", , "
        parts = details.split(', , ')
        cleaned_parts = []
        for part in parts:
            # Extract the layover time and airport details
            time, airport = part.split(' layover, <b>')
            airport = airport.replace('</b>', '')
            cleaned_parts.append((time.strip(), airport.strip()))
        return cleaned_parts
    else:
        return []

# Apply the function to the 'Stopover Details' column
df['Cleaned Stopover Details'] = df['Stopover Details'].apply(clean_split_stopover)

# Convert the cleaned details into separate columns
stopover_df = df['Cleaned Stopover Details'].apply(pd.Series)

# Extract layover time and airport into separate columns
flat_df = pd.DataFrame()

for col in stopover_df.columns:
    temp_list = stopover_df[col].apply(lambda x: x if isinstance(x, tuple) else (np.nan, np.nan))
    temp_df = pd.DataFrame(temp_list.tolist(), columns=[f'Stopover_{col+1}_Time', f'Stopover_{col+1}_Airport'])
    flat_df = pd.concat([flat_df, temp_df], axis=1)

# Remove leading commas in Stopover_1_Time
flat_df['Stopover_1_Time'] = flat_df['Stopover_1_Time'].str.replace('^, ', '', regex=True)

# Function to convert time string to minutes
def time_to_minutes(time_str):
    if pd.isna(time_str):
        return 0
    hours, minutes = 0, 0
    if 'h' in time_str:
        hours = int(time_str.split('h')[0].strip())
        time_str = time_str.split('h')[1].strip()
    if 'm' in time_str:
        minutes = int(time_str.split('m')[0].strip())
    return hours * 60 + minutes

# Calculate total stopover time
flat_df['Total_Stopover_Time'] = flat_df[[col for col in flat_df.columns if 'Time' in col]].applymap(time_to_minutes).sum(axis=1)

# Convert total time back to hours and minutes
def minutes_to_time(minutes):
    hours = minutes // 60
    minutes = minutes % 60
    return f'{hours}h {minutes}m'

flat_df['Total_Stopover_Time'] = flat_df['Total_Stopover_Time'].apply(minutes_to_time)

# Show the final DataFrame
print(flat_df)

/var/folders/6c/n8sfjh_d78lfj_kyk83qgvy00000gn/T/ipykernel_17659/1600515772.py:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  flat_df['Total_Stopover_Time'] = flat_df[[col for col in flat_df.columns if 'Time' in col]].applymap(time_to_minutes).sum(axis=1)


       Stopover_1_Time  Stopover_1_Airport Stopover_2_Time  \
0              19h 25m  Amsterdam Schiphol          4h 45m   
1              20h 35m  Amsterdam Schiphol          3h 25m   
2              22h 20m  Amsterdam Schiphol          1h 40m   
3               4h 55m  Amsterdam Schiphol         16h 05m   
4               2h 10m  Amsterdam Schiphol         18h 45m   
...                ...                 ...             ...   
207011          3h 35m    Addis Ababa Bole          5h 25m   
207012          3h 35m    Addis Ababa Bole          4h 25m   
207013          3h 35m    Addis Ababa Bole          9h 55m   
207014          3h 35m    Addis Ababa Bole          5h 25m   
207015          3h 35m    Addis Ababa Bole          9h 55m   

             Stopover_2_Airport Stopover_3_Time Stopover_3_Airport  \
0               London Heathrow             NaN                NaN   
1               London Heathrow             NaN                NaN   
2               London Heathrow             N

In [9]:
flat_df.head()

,Stopover_1_Time,Stopover_1_Airport,Stopover_2_Time,Stopover_2_Airport,Stopover_3_Time,Stopover_3_Airport,Total_Stopover_Time
0,19h 25m,Amsterdam Schiphol,4h 45m,London Heathrow,NaN,NaN,24h 10m
1,20h 35m,Amsterdam Schiphol,3h 25m,London Heathrow,NaN,NaN,24h 0m
2,22h 20m,Amsterdam Schiphol,1h 40m,London Heathrow,NaN,NaN,24h 0m
3,4h 55m,Amsterdam Schiphol,16h 05m,Paris Charles de Gaulle,NaN,NaN,21h 0m
4,2h 10m,Amsterdam Schiphol,18h 45m,Paris Charles de Gaulle,NaN,NaN,20h 55m


In [11]:
# Combine flat_df with the original df
df = pd.concat([df, flat_df], axis=1)

In [13]:
df.head()

,Airline,Source,Destination,Departure,Arrival,Number of Stops,Stopover Details,Price,Class,Date,Cleaned Stopover Details,Stopover_1_Time,Stopover_1_Airport,Stopover_2_Time,Stopover_2_Airport,Stopover_3_Time,Stopover_3_Airport,Total_Stopover_Time
0,"Delta • Operated by Virgin Atlantic, KLM",Toronto Pearson Intl,Bengaluru Intl,10:40 pm,2:45 am+3,2 stops,", 19h 25m layover, <b>Amsterdam Schiphol</b>, ...","C$ 2,229",Main Cabin,2024-06-22,"[(, 19h 25m, Amsterdam Schiphol), (4h 45m, Lon...",19h 25m,Amsterdam Schiphol,4h 45m,London Heathrow,NaN,NaN,24h 10m
1,"Delta • Operated by Virgin Atlantic, KLM",Toronto Pearson Intl,Bengaluru Intl,10:40 pm,2:45 am+3,2 stops,", 20h 35m layover, <b>Amsterdam Schiphol</b>, ...","C$ 2,229",Main Cabin,2024-06-22,"[(, 20h 35m, Amsterdam Schiphol), (3h 25m, Lon...",20h 35m,Amsterdam Schiphol,3h 25m,London Heathrow,NaN,NaN,24h 0m
2,"Delta • Operated by Virgin Atlantic, KLM",Toronto Pearson Intl,Bengaluru Intl,10:40 pm,2:45 am+3,2 stops,", 22h 20m layover, <b>Amsterdam Schiphol</b>, ...","C$ 2,524",Main Cabin,2024-06-22,"[(, 22h 20m, Amsterdam Schiphol), (1h 40m, Lon...",22h 20m,Amsterdam Schiphol,1h 40m,London Heathrow,NaN,NaN,24h 0m
3,"Delta • Operated by KLM, Air France",Toronto Pearson Intl,Bengaluru Intl,10:40 pm,11:55 pm+2,2 stops,", 4h 55m layover, <b>Amsterdam Schiphol</b>, ,...","C$ 3,690",Main Cabin,2024-06-22,"[(, 4h 55m, Amsterdam Schiphol), (16h 05m, Par...",4h 55m,Amsterdam Schiphol,16h 05m,Paris Charles de Gaulle,NaN,NaN,21h 0m
4,"Delta • Operated by KLM, Air France",Toronto Pearson Intl,Bengaluru Intl,10:40 pm,11:55 pm+2,2 stops,", 2h 10m layover, <b>Amsterdam Schiphol</b>, ,...","C$ 3,690",Main Cabin,2024-06-22,"[(, 2h 10m, Amsterdam Schiphol), (18h 45m, Par...",2h 10m,Amsterdam Schiphol,18h 45m,Paris Charles de Gaulle,NaN,NaN,20h 55m


In [17]:
df = df.drop(columns=['Cleaned Stopover Details','Stopover Details'])

In [19]:
df.head()

,Airline,Source,Destination,Departure,Arrival,Number of Stops,Price,Class,Date,Stopover_1_Time,Stopover_1_Airport,Stopover_2_Time,Stopover_2_Airport,Stopover_3_Time,Stopover_3_Airport,Total_Stopover_Time
0,"Delta • Operated by Virgin Atlantic, KLM",Toronto Pearson Intl,Bengaluru Intl,10:40 pm,2:45 am+3,2 stops,"C$ 2,229",Main Cabin,2024-06-22,19h 25m,Amsterdam Schiphol,4h 45m,London Heathrow,NaN,NaN,24h 10m
1,"Delta • Operated by Virgin Atlantic, KLM",Toronto Pearson Intl,Bengaluru Intl,10:40 pm,2:45 am+3,2 stops,"C$ 2,229",Main Cabin,2024-06-22,20h 35m,Amsterdam Schiphol,3h 25m,London Heathrow,NaN,NaN,24h 0m
2,"Delta • Operated by Virgin Atlantic, KLM",Toronto Pearson Intl,Bengaluru Intl,10:40 pm,2:45 am+3,2 stops,"C$ 2,524",Main Cabin,2024-06-22,22h 20m,Amsterdam Schiphol,1h 40m,London Heathrow,NaN,NaN,24h 0m
3,"Delta • Operated by KLM, Air France",Toronto Pearson Intl,Bengaluru Intl,10:40 pm,11:55 pm+2,2 stops,"C$ 3,690",Main Cabin,2024-06-22,4h 55m,Amsterdam Schiphol,16h 05m,Paris Charles de Gaulle,NaN,NaN,21h 0m
4,"Delta • Operated by KLM, Air France",Toronto Pearson Intl,Bengaluru Intl,10:40 pm,11:55 pm+2,2 stops,"C$ 3,690",Main Cabin,2024-06-22,2h 10m,Amsterdam Schiphol,18h 45m,Paris Charles de Gaulle,NaN,NaN,20h 55m


In [23]:
# Split the 'Airline' column at '• Operated by', handle missing parts by filling with None
split_df = df['Airline'].str.split('• Operated by', n=1, expand=True)

# Assign split data to new columns and fill missing values
df['Airline'] = split_df[0].str.strip()
df['Operated'] = split_df[1].str.strip().fillna('None')

# Display the resulting DataFrame
print(df)

                          Airline                    Source  \
0                           Delta      Toronto Pearson Intl   
1                           Delta      Toronto Pearson Intl   
2                           Delta      Toronto Pearson Intl   
3                           Delta      Toronto Pearson Intl   
4                           Delta      Toronto Pearson Intl   
...                           ...                       ...   
207011          Multiple Airlines  Kano Aminu Kano Intl Apt   
207012  Ethiopian Air, Air Canada  Kano Aminu Kano Intl Apt   
207013          Multiple Airlines  Kano Aminu Kano Intl Apt   
207014          Multiple Airlines  Kano Aminu Kano Intl Apt   
207015          Multiple Airlines  Kano Aminu Kano Intl Apt   

                 Destination Departure     Arrival Number of Stops      Price  \
0             Bengaluru Intl  10:40 pm   2:45 am+3         2 stops   C$ 2,229   
1             Bengaluru Intl  10:40 pm   2:45 am+3         2 stops   C$ 2,229   


In [25]:
# Remove the 'C$' prefix and commas, and convert to numeric
df['price in CAD'] = df['Price'].str.replace('C$', '').str.replace(',', '').astype(int)

# Drop the old 'Price' column
df = df.drop(columns=['Price'])

In [27]:
df.head()

,Airline,Source,Destination,Departure,Arrival,Number of Stops,Class,Date,Stopover_1_Time,Stopover_1_Airport,Stopover_2_Time,Stopover_2_Airport,Stopover_3_Time,Stopover_3_Airport,Total_Stopover_Time,Operated,price in CAD
0,Delta,Toronto Pearson Intl,Bengaluru Intl,10:40 pm,2:45 am+3,2 stops,Main Cabin,2024-06-22,19h 25m,Amsterdam Schiphol,4h 45m,London Heathrow,NaN,NaN,24h 10m,"Virgin Atlantic, KLM",2229
1,Delta,Toronto Pearson Intl,Bengaluru Intl,10:40 pm,2:45 am+3,2 stops,Main Cabin,2024-06-22,20h 35m,Amsterdam Schiphol,3h 25m,London Heathrow,NaN,NaN,24h 0m,"Virgin Atlantic, KLM",2229
2,Delta,Toronto Pearson Intl,Bengaluru Intl,10:40 pm,2:45 am+3,2 stops,Main Cabin,2024-06-22,22h 20m,Amsterdam Schiphol,1h 40m,London Heathrow,NaN,NaN,24h 0m,"Virgin Atlantic, KLM",2524
3,Delta,Toronto Pearson Intl,Bengaluru Intl,10:40 pm,11:55 pm+2,2 stops,Main Cabin,2024-06-22,4h 55m,Amsterdam Schiphol,16h 05m,Paris Charles de Gaulle,NaN,NaN,21h 0m,"KLM, Air France",3690
4,Delta,Toronto Pearson Intl,Bengaluru Intl,10:40 pm,11:55 pm+2,2 stops,Main Cabin,2024-06-22,2h 10m,Amsterdam Schiphol,18h 45m,Paris Charles de Gaulle,NaN,NaN,20h 55m,"KLM, Air France",3690


In [29]:
df.shape

(207016, 17)

In [ ]:
# df.to_csv('Rehan.csv', index=False)

In [43]:
from datetime import datetime, timedelta

# Function to calculate days_left
def calculate_days_left(flight_date):
    start_date = datetime(2024, 6, 22)
    current_date = datetime.strptime(flight_date, '%Y-%m-%d')
    return (current_date - start_date).days + 1

# Apply the function to create the new column
df['days_left'] = df['Date'].apply(calculate_days_left)

# Display the updated dataframe
df.head()


,Airline,Source,Destination,Departure,Arrival,Number of Stops,Class,Date,Stopover_1_Time,Stopover_1_Airport,Stopover_2_Time,Stopover_2_Airport,Stopover_3_Time,Stopover_3_Airport,Total_Stopover_Time,Operated,price in CAD,days_left
0,Delta,Toronto Pearson Intl,Bengaluru Intl,10:40 pm,2:45 am+3,2 stops,Main Cabin,2024-06-22,19h 25m,Amsterdam Schiphol,4h 45m,London Heathrow,NaN,NaN,24h 10m,"Virgin Atlantic, KLM",2229,1
1,Delta,Toronto Pearson Intl,Bengaluru Intl,10:40 pm,2:45 am+3,2 stops,Main Cabin,2024-06-22,20h 35m,Amsterdam Schiphol,3h 25m,London Heathrow,NaN,NaN,24h 0m,"Virgin Atlantic, KLM",2229,1
2,Delta,Toronto Pearson Intl,Bengaluru Intl,10:40 pm,2:45 am+3,2 stops,Main Cabin,2024-06-22,22h 20m,Amsterdam Schiphol,1h 40m,London Heathrow,NaN,NaN,24h 0m,"Virgin Atlantic, KLM",2524,1
3,Delta,Toronto Pearson Intl,Bengaluru Intl,10:40 pm,11:55 pm+2,2 stops,Main Cabin,2024-06-22,4h 55m,Amsterdam Schiphol,16h 05m,Paris Charles de Gaulle,NaN,NaN,21h 0m,"KLM, Air France",3690,1
4,Delta,Toronto Pearson Intl,Bengaluru Intl,10:40 pm,11:55 pm+2,2 stops,Main Cabin,2024-06-22,2h 10m,Amsterdam Schiphol,18h 45m,Paris Charles de Gaulle,NaN,NaN,20h 55m,"KLM, Air France",3690,1


In [35]:
df.to_csv('Final3.csv', index=False)